#Project

##prepare

In [ ]:
!pip install tensorflow-datasets
!pip3 install datasets
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments
from datasets import Dataset
from datasets import load_dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re
import torch
import transformers

In [ ]:
dataset = load_dataset("zeroshot/twitter-financial-news-sentiment")

dataset_dict = dataset['train']

df_train = dataset_dict.to_pandas()

dataset_dict = dataset['validation']

df_test = dataset_dict.to_pandas()

print(df_train.head())
print(df_test.head())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

                                                text  label
0  $BYND - JPMorgan reels in expectations on Beyo...      0
1  $CCL $RCL - Nomura points to bookings weakness...      0
2  $CX - Cemex cut at Credit Suisse, J.P. Morgan ...      0
3  $ESS: BTIG Research cuts to Neutral https://t....      0
4  $FNKO - Funko slides after Piper Jaffray PT cu...      0
                                                text  label
0  $ALLY - Ally Financial pulls outlook https://t...      0
1  $DELL $HPE - Dell, HPE targets trimmed on comp...      0
2  $PRTY - Moody's turns negative on Party City h...      0
3                   $SAN: Deutsche Bank cuts to Hold      0
4                  $SITC: Compass Point cuts to Sell      0


In [ ]:
twitter_sentiments = {
    "LABEL_0": "Bearish",
    "LABEL_1": "Bullish",
    "LABEL_2": "Neutral"
}

In [ ]:
print('train:')
print(df_train['label'].value_counts())
print('tesst:')
print(df_test['label'].value_counts())

train:
2    6178
1    1923
0    1442
Name: label, dtype: int64
tesst:
2    1566
1     475
0     347
Name: label, dtype: int64


## Balancing and processing the data

In [ ]:
df_train = pd.concat([df_train,df_test],ignore_index=True)
df_train['label'].value_counts()

2    7744
1    2398
0    1789
Name: label, dtype: int64

In [ ]:
# !pip install demoji
# import demoji

In [ ]:
def preprocess_text(text):

    text = re.sub(r'http[s]?://\S+', '', text)
    #text = demoji.replace(text, '')
    #text = re.sub(r'\b(?:#|@|\$)\w+\b', '', text)
    #text = text.replace('-', '')

    return text

In [ ]:
df_train['text'] = df_train['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [ ]:
# def get_len(tweet):
#   lst = tweet.split()
#   return len(lst)

In [ ]:
# import matplotlib.pyplot as plt

# df_train['word_count'] = df_train['text'].apply(get_len)

# plt.figure(figsize=(10, 6))
# plt.hist(df_train['word_count'], bins=range(min(df_train['word_count']), max(df_train['word_count']) + 2), color='skyblue', edgecolor='black')
# plt.xlabel('Number of Words')
# plt.ylabel('Number of Tweets')
# plt.title('Distribution of Number of Words in Tweets')
# plt.xticks(range(min(df_train['word_count']), max(df_train['word_count']) + 1))
# plt.grid(axis='y')

# # Show the plot
# plt.show()

In [ ]:
df_train.label.value_counts()

2    7744
1    2398
0    1789
Name: label, dtype: int64

In [ ]:
df_2 = df_train[df_train['label']==2][:2300].copy()
df_01 = df_train[df_train['label']!=2].copy()
df_train = pd.concat([df_01, df_2],ignore_index=True)
df_train['label'].value_counts()

1    2398
2    2300
0    1789
Name: label, dtype: int64

In [ ]:
df_train.label.value_counts()

1    2398
2    2300
0    1789
Name: label, dtype: int64

In [ ]:
df_train, df_test = train_test_split(df_train, stratify=df_train['label'],test_size=0.1, random_state=42)

In [ ]:
# df = pd.read_csv('https://raw.githubusercontent.com/gabirayman/more_financial_tweets/main/tweets_labelled_09042020_16072020.csv',sep = ";")
# df = df[df.sentiment.notna()]
# df = df[['text','sentiment']]
# df.columns = ['text', 'label']
# df['text'] = df['text'].apply(preprocess_text)
# df['len'] = df['text'].apply(get_len)
# df = df[df.len > 6]

# label_mapping = {'negative': 0, 'positive': 1, 'neutral': 2}
# df['label'] = df['label'].map(label_mapping)
# df_train = pd.concat([df_train, df], ignore_index=True)

In [ ]:
print(df_train['label'].value_counts())
print(df_test['label'].value_counts())

1    2158
2    2070
0    1610
Name: label, dtype: int64
1    240
2    230
0    179
Name: label, dtype: int64


In [ ]:
df_train, df_val = train_test_split(df_train, stratify=df_train['label'],test_size=0.1, random_state=42)

In [ ]:
torch.cuda.is_available()

True

##ROBERTA Model

In [ ]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

In [ ]:
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Assuming df_train, df_val, and df_test are your dataframes
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)

# Tokenize datasets
dataset_train = dataset_train.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_val = dataset_val.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test = dataset_test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)

# Set dataset format for PyTorch
dataset_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
dataset_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
dataset_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Define compute_metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': accuracy_score(predictions, labels)}

# Training arguments
args = TrainingArguments(
    output_dir='temp/',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)

# Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

Map:   0%|          | 0/5254 [00:00<?, ? examples/s]

Map:   0%|          | 0/584 [00:00<?, ? examples/s]

Map:   0%|          | 0/649 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.338571,0.869863
2,No log,0.296863,0.892123
3,No log,0.346401,0.883562
4,0.332400,0.295770,0.912671


TrainOutput(global_step=660, training_loss=0.28176334554498844, metrics={'train_runtime': 176.7614, 'train_samples_per_second': 118.895, 'train_steps_per_second': 3.734, 'total_flos': 1382397896742912.0, 'train_loss': 0.28176334554498844, 'epoch': 4.0})

In [ ]:
pred = trainer.predict(dataset_test).predictions
final_verdict = np.argmax(pred, axis=1)
real = df_test['label'].tolist()
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


# Assuming 'y_true' is the true target values and 'y_pred' is the predicted values
# Replace them with your actual variable names

y_true = real
y_pred = list(final_verdict)

# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Compute precision, recall, and F1 score for each class
precision = precision_score(y_true, y_pred, average=None)
recall = recall_score(y_true, y_pred, average=None)
f1 = f1_score(y_true, y_pred, average=None)

# Print precision, recall, and F1 score for each class
for i in range(len(precision)):
    print(f'Class {i}: Precision={precision[i]:.4f}, Recall={recall[i]:.4f}, F1 Score={f1[i]:.4f}')

# Compute and print confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('\nConfusion Matrix:')
print(conf_matrix)

# Print classification report
class_report = classification_report(y_true, y_pred)
print('\nClassification Report:')
print(class_report)

Accuracy: 0.9076
Class 0: Precision=0.9076, Recall=0.9330, F1 Score=0.9201
Class 1: Precision=0.9053, Recall=0.9167, F1 Score=0.9110
Class 2: Precision=0.9099, Recall=0.8783, F1 Score=0.8938

Confusion Matrix:
[[167   6   6]
 [  6 220  14]
 [ 11  17 202]]

Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       179
           1       0.91      0.92      0.91       240
           2       0.91      0.88      0.89       230

    accuracy                           0.91       649
   macro avg       0.91      0.91      0.91       649
weighted avg       0.91      0.91      0.91       649



In [ ]:
pred = trainer.predict(dataset_test).predictions
final_verdict = np.argmax(pred, axis=1)
real = df_test['label'].tolist()
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


# Assuming 'y_true' is the true target values and 'y_pred' is the predicted values
# Replace them with your actual variable names

y_true = real
y_pred = list(final_verdict)

# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Compute precision, recall, and F1 score for each class
precision = precision_score(y_true, y_pred, average=None)
recall = recall_score(y_true, y_pred, average=None)
f1 = f1_score(y_true, y_pred, average=None)

# Print precision, recall, and F1 score for each class
for i in range(len(precision)):
    print(f'Class {i}: Precision={precision[i]:.4f}, Recall={recall[i]:.4f}, F1 Score={f1[i]:.4f}')

# Compute and print confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('\nConfusion Matrix:')
print(conf_matrix)

# Print classification report
class_report = classification_report(y_true, y_pred)
print('\nClassification Report:')
print(class_report)

Accuracy: 0.9091
Class 0: Precision=0.8836, Recall=0.9330, F1 Score=0.9076
Class 1: Precision=0.9020, Recall=0.9208, F1 Score=0.9113
Class 2: Precision=0.9395, Recall=0.8783, F1 Score=0.9079

Confusion Matrix:
[[167   8   4]
 [ 10 221   9]
 [ 12  16 202]]

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.93      0.91       179
           1       0.90      0.92      0.91       240
           2       0.94      0.88      0.91       230

    accuracy                           0.91       649
   macro avg       0.91      0.91      0.91       649
weighted avg       0.91      0.91      0.91       649



In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import create_repo
create_repo("GabiRayman/Financial_Tweet_Class_Model")

RepoUrl('https://huggingface.co/GabiRayman/Financial_Tweet_Class_Model', endpoint='https://huggingface.co', repo_type='model', repo_id='GabiRayman/Financial_Tweet_Class_Model')

In [ ]:
model.push_to_hub("GabiRayman/Financial_Tweet_Class_Model")
tokenizer.push_to_hub("GabiRayman/Financial_Tweet_Class_Model")

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/GabiRayman/Financial_Tweet_Class_Model/commit/cf02856037eb73173b8a9e268dd7d981fc358a4d', commit_message='Upload tokenizer', commit_description='', oid='cf02856037eb73173b8a9e268dd7d981fc358a4d', pr_url=None, pr_revision=None, pr_num=None)

#How to use the finetuned model from HF

In [ ]:
model_name = "GabiRayman/Financial_Tweet_Class_Model"
tokenizer_name = "GabiRayman/Financial_Tweet_Class_Model"

model_test = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3) #, num_labels=2)
tokenizer_test = RobertaTokenizer.from_pretrained(tokenizer_name)

config.json:   0%|          | 0.00/889 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

In [ ]:
to_pred = df_test.text.tolist()
tokenized_texts = tokenizer_test(to_pred, return_tensors="pt", truncation=True, padding='max_length', max_length=128) #, padding=True ,return_tensors="pt"
with torch.no_grad():
    outputs = model_test(**tokenized_texts)

probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
pred = torch.argmax(probabilities,dim=1).tolist()

In [ ]:
y_true = df_test.label.to_list()
y_pred = list(pred)

# Compute accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Compute precision, recall, and F1 score for each class
precision = precision_score(y_true, y_pred, average=None)
recall = recall_score(y_true, y_pred, average=None)
f1 = f1_score(y_true, y_pred, average=None)

# Print precision, recall, and F1 score for each class
for i in range(len(precision)):
    print(f'Class {i}: Precision={precision[i]:.4f}, Recall={recall[i]:.4f}, F1 Score={f1[i]:.4f}')

# Compute and print confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('\nConfusion Matrix:')
print(conf_matrix)

# Print classification report
class_report = classification_report(y_true, y_pred)
print('\nClassification Report:')
print(class_report)

Accuracy: 0.9091
Class 0: Precision=0.8836, Recall=0.9330, F1 Score=0.9076
Class 1: Precision=0.9020, Recall=0.9208, F1 Score=0.9113
Class 2: Precision=0.9395, Recall=0.8783, F1 Score=0.9079

Confusion Matrix:
[[167   8   4]
 [ 10 221   9]
 [ 12  16 202]]

Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.93      0.91       179
           1       0.90      0.92      0.91       240
           2       0.94      0.88      0.91       230

    accuracy                           0.91       649
   macro avg       0.91      0.91      0.91       649
weighted avg       0.91      0.91      0.91       649

